In [1]:
#импорт библиотек, загрузка модели, весов, разметки клаассов
import numpy as np
import pandas as pd

In [19]:
#импорт описания классов (выдрал с альта софта)
dscr = pd.read_csv("../data/desc.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})

In [2]:
#импорт последнего датасета (со спарсенными)
df = pd.read_csv("../data/mergedcleared0407.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})

In [5]:
df.shape

(261080, 2)

In [4]:
#убираем классы, которых меньше 10 экземпляров
counts = df.value_counts('id')
df_count = pd.DataFrame(data={'id': counts.index, 'count': counts.values})
df = df[~df['id'].isin(df_count[df_count['count']<10]['id'])]

In [5]:
#функция для вывода обучающих данных по коду. При pandas=False возвращает numpy массив
def whats_data(id, pandas=True):
    if pandas == True:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)]
    else:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)].to_numpy()

In [6]:
#импорт наивного байеса
from sklearn.naive_bayes import ComplementNB, MultinomialNB, BernoulliNB, CategoricalNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from sklearn.model_selection import GridSearchCV, cross_val_score

In [7]:
#делаем вокабуляр для веркоров предложений
vect = CountVectorizer(binary=True)
vect.fit(df['label'])
len(vect.vocabulary_)

254935

In [8]:
#делим на тест и трэйн
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['id'], shuffle=True)

In [9]:
#получаем векторы предложений
X_train = vect.transform(df_train['label'])
X_test = vect.transform(df_test['label'])

In [10]:
#обучаем всю эту ситуацию
clf = MultinomialNB().fit(X_train, df_train['id'])

In [11]:
#смотрим предсказания на тесте
predicts = clf.predict(X_test)
print(classification_report(df_test['id'], predicts))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

  0207149909       1.00      1.00      1.00        14
  0304999800       1.00      0.67      0.80        12
  0305542000       1.00      0.56      0.71         9
  0305549000       0.79      0.92      0.85        12
  0305599000       0.67      0.57      0.62        14
  0604909900       1.00      0.93      0.96        14
  0701905000       0.94      0.94      0.94        18
  0703200000       0.80      0.57      0.67        14
  0711400000       1.00      0.88      0.93         8
  0711590000       0.00      0.00      0.00         3
  0712909000       0.88      0.50      0.64        14
  0713339000       0.91      0.91      0.91        11
  0801110000       1.00      0.65      0.79        17
  0802310000       1.00      0.50      0.67        22
  0802320000       0.37      0.97      0.54        34
  0803901000       1.00      0.58      0.74        12
  0804300009       1.00      0.54      0.70        13
  0804500001       0.82    

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
#Как пользоваться:
#Передаем методу трансформ список текстовых описаний, по которым хотим получить классы. 
#Получаем векторы предложений...

txt_to_vector = vect.transform(['орех грецкий', 'автопокрышка', 'труба'])

In [38]:
#скармливаем векторы модели, получаем предсказания

result = clf.predict(txt_to_vector)

In [39]:
#Читаем, что получили

for each in result:
    #print(each[:4])
    print(each, dscr[dscr['id'] == each[:4]].iloc[0]['label'])

0802320000 Прочие орехи, свежие или сушеные, очищенные от скорлупы или неочищенные, с кожурой или без кожуры
4011100009 Шины и покрышки пневматические резиновые новые
8708999709 Части и принадлежности моторных транспортных средств товарных позиций 8701 - 8705
